In [162]:
import numpy as np
from scipy.signal import spectrogram, lfilter
from scipy.signal.windows import hann
from scipy.fftpack import dct
import os
from matplotlib import pyplot as plt
import cv2
import librosa
import time

In [163]:
#setting eps value for converting spec to log
eps = 1e-10

In [164]:
'''
def compute_aci(spec, time_steps, freq_bins):
      
    #Function computes Acoustic Complexity Index
    #According to:
    #http://www.iinsteco.org/people/pubblications/almo/2011/2011_A_new%20methodology_to_infer_the_singing_activity.pdf
    print(time_steps, freq_bins)
    ACI_freq_bins = np.zeros((freq_bins, 1))
    ACI_freq_bins_norm = np.zeros((freq_bins, 1))

    ACI_all = 0
    
    for i in range(freq_bins):
        
        #for loop for all time steps in same freq bin - sum each row
        for j in range(time_steps-1):
            abs_diff = np.abs( spec[i, j+1] - spec[i,j] )
            ACI_freq_bins[i] = abs_diff+ ACI_freq_bins[i] 
            
            #if( (i==0) and ((j==0) or (j==1))):
                #print(abs_diff, ACI_freq_bins[i])
        
        #normalization
        row_sum = np.sum(spec[i,:])
        #print("Row sum = ", str(row_sum), "and abs sum is ", ACI_freq_bins[i])
        ACI_freq_bins_norm[i] = ACI_freq_bins[i]/row_sum
        print(ACI_freq_bins_norm[i])
            
    ACI_all = np.sum(ACI_freq_bins_norm)
   
    return ACI_all
'''

'\ndef compute_aci(spec, time_steps, freq_bins):\n      \n    #Function computes Acoustic Complexity Index\n    #According to:\n    #http://www.iinsteco.org/people/pubblications/almo/2011/2011_A_new%20methodology_to_infer_the_singing_activity.pdf\n    print(time_steps, freq_bins)\n    ACI_freq_bins = np.zeros((freq_bins, 1))\n    ACI_freq_bins_norm = np.zeros((freq_bins, 1))\n\n    ACI_all = 0\n    \n    for i in range(freq_bins):\n        \n        #for loop for all time steps in same freq bin - sum each row\n        for j in range(time_steps-1):\n            abs_diff = np.abs( spec[i, j+1] - spec[i,j] )\n            ACI_freq_bins[i] = abs_diff+ ACI_freq_bins[i] \n            \n            #if( (i==0) and ((j==0) or (j==1))):\n                #print(abs_diff, ACI_freq_bins[i])\n        \n        #normalization\n        row_sum = np.sum(spec[i,:])\n        #print("Row sum = ", str(row_sum), "and abs sum is ", ACI_freq_bins[i])\n        ACI_freq_bins_norm[i] = ACI_freq_bins[i]/row_sum\n

In [165]:
#'''
def compute_aci(specs, time_steps, freq_bins):
      
    #Function computes Acoustic Complexity Index
    #According to:
    #http://www.iinsteco.org/people/pubblications/almo/2011/2011_A_new%20methodology_to_infer_the_singing_activity.pdf

    #print("Dimensions are ", specs.shape)
      
    #abs_diff = np.abs(specs[:, :, 2:] - specs[:, :, 1:-1])
    #ACI = np.sum(specs.sum(axis=2) / abs_diff.sum(axis=2), axis=1)
    
    abs_diff = np.abs(specs[:, :, 1:] - specs[:, :, 0:time_steps-1])
    
    #row_sums = np.sum(specs, axis = 2)
    #print(row_sums[0][0])
    
    #abs_sums = np.sum(abs_diff, axis = 2)
    #print(abs_sums[0][0])
    
    #ACI_vector = abs_sums/row_sums
    
    
    ACI = np.sum(np.sum(abs_diff, axis = 2)/np.sum(specs, axis = 2), axis = 1)
    #print(ACI_vector[0][0])
    
    return ACI
 #   '''

In [166]:
#function to load spectrograms and display dimensions (optional)
def load_display_dimensions(class_name, dim=0):

    #   description of data files
    #   spec_data is like a dict with fields
    #   'specs' (contains the mel-filtered spectrograms, in linear scale, as [num_specs, num_rows, num_cols])
    #   'spec_f' (contains the frequency axis points as a 1D array)
    #   'spec_t' (contains the time axis points as a 1D array)   
    
    #choose if _spec or _melspec
    fname_load = class_name + '_spec' + '.npz'
    spec_data = np.load(fname_load)
    
    time_steps = len(spec_data['spec_t'])
    freq_bins = len(spec_data['spec_f'])
    
    dim = 0
    
    if (dim==1):
        print('File has %i specs of dimensions (%i x %i)' % (
        spec_data['specs'].shape[0], spec_data['specs'].shape[1], spec_data['specs'].shape[2]))
        print('Frequency axis points are in spec_data[\'spec_f\'] and has %i values' % freq_bins)
        print('Time axis points are in spec_data[\'spec_t\'] and has %i values' % time_steps)
        
    return spec_data, time_steps, freq_bins

In [167]:
#function to calculate all acoustic indices and plot spectrogram (optional)
def calc_plot(spec_start_idx, num_specs, spec_data, time_steps, freq_bins, freq_vals, plot=0):

    ACI = np.zeros((num_specs, 1))
        
    #Calculation of acoustic indices for num_spec spectrograms here
    all_specs = 10 * np.log10(spec_data['specs'][spec_start_idx:spec_start_idx+num_specs, :, :] + eps)
    ACI = compute_aci(all_specs, time_steps, freq_bins)
      
    for spec_idx in range(num_specs):
     
        #Getting current spec from saved log spec
        current_spec = all_specs[spec_start_idx + spec_idx, :, :] 
        
        #ACI[spec_idx] = compute_aci(current_spec, time_steps, freq_bins)
        
        if(plot==1):
            cax = axes[spec_idx].imshow(current_spec, cmap=plt.get_cmap('jet'), interpolation='none', origin='lower')
            fig.colorbar(cax, ax=axes[spec_idx], orientation='vertical')
        
    
    return ACI
       

In [168]:
def calls(class_name, spec_start_idx, num_specs, multiples, num_bins, dim=0, plot=0):

    #Load the correct file
    spec_data, time_steps, freq_bins = load_display_dimensions(class_name, dim)
    
    plot = 0
    #Computing acoustic indices
    ACI = calc_plot(spec_start_idx, num_specs, spec_data, time_steps, freq_bins, spec_data['spec_f'], plot)
    #2. Roughness 3. Temporal entropy 4. Acoustic diversity 5. Acoustic richness 
    #6. Spectral diversity 7. Spectral kurtosis 8. Spectral entropy 9. ?? 10. ??
    
    return ACI

In [169]:
#environmental sounds/noise
fname_list = ["Airplane_Sound", "Heavy_rain", "Brown_Noise", "Pink_Noise", "White_Noise", "Rufous_Antpitta", "Grey_headed_woodpecker", "Italian_Sparrow", "Hawk_scream", "Dove"]



In [170]:
#Main

#start timer
tic = time.time()
  
#Make sure the indices don't get larger than spec_data['specs'].shape[0]
spec_start_idx = 0
#num_specs = 2

num_specs = 1

#Other parameters
plot = 0
dim = 1
num_bins = 8            #for compute_adi_even
multiples = 1000

ACI = np.zeros((10, 1))
#Calls for each class

for i in range(10):
    
    #str_name = 'fname' + str(i+1)
    ACI[i] = calls(fname_list[i], spec_start_idx, num_specs, multiples, num_bins, dim, plot)
    
    print("For ", fname_list[i], "the ACI value is: ", ACI[i])

#end timer
toc = time.time()
print("Time taken: " + str(1000*(toc-tic)) + " ms")

#Display    
#plt.show()

For  Airplane_Sound the ACI value is:  [-36.55999756]
For  Heavy_rain the ACI value is:  [-37.42383575]
For  Brown_Noise the ACI value is:  [-32.15776062]
For  Pink_Noise the ACI value is:  [-42.99048996]
For  White_Noise the ACI value is:  [-42.99048996]
For  Rufous_Antpitta the ACI value is:  [-26.73933792]
For  Grey_headed_woodpecker the ACI value is:  [-28.14480591]
For  Italian_Sparrow the ACI value is:  [-28.90440178]
For  Hawk_scream the ACI value is:  [-31.76561356]
For  Dove the ACI value is:  [-25.44630051]
Time taken: 191.52402877807617 ms
